In [8]:
!pip install flask
!pip install numpy
!pip install pymongo
!pip install paho-mqtt

In [9]:
#load some virtual_environment, request, jsonify
from flask import Flask, render_template, request, jsonify
import numpy as np
from pymongo import MongoClient
import paho.mqtt.client as mqtt
import threading
import json

In [10]:
app = Flask(__name__)
client = MongoClient('localhost',27017)
db = client.flask_database
sensor_data_collection = db.sensor_data


In [11]:
VALID_KEYS={"TDS","TURBIDITY","PH"}
MQTT_BROKER = "localhost"
MQTT_PORT = 1883
MQTT_TOPIC = "quality"

In [12]:
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(MQTT_TOPIC)

def on_message(client, userdata, msg):
    print(f"Message recieved: {msg.topic} -> {msg.payload.decode()}")

    try:
        data = json.loads(msg.payload.decode())
        if isinstance(data, dict):
            if all(key in data for key in VALID_KEYS):
                filtered_data = {key: data[key] for key in VALID_KEYS}
                sensor_data_collection.insert_one(filtered_data)    
                print("Data inserted into MongoDB:", filtered_data)
            else:
                print("Invalid keys in data")
        else:
            print("Invalid data format")
    except Exception as e:
        print("Error parsing data:", e)

def start_mqtt():
    mqtt_client = mqtt.Client()
    mqtt_client.on_connect = on_connect
    mqtt_client.on_message = on_message
    mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
    mqtt_client.loop_forever()

    #Flask routes

@app.route("/", methods=["GET"])
def index():
    return render_template("index.html")

@app.route("/data", methods=["GET"])
def get_data():
    data = list(sensor_data_collection.find({}, {"_id": 0}))
    return jsonify({"sensor_data":data})

@app.route("/test_db", methods=["GET"])
def test_db():
    try:
        db.admin.command('ping')
        return jsonify({"status":"MongoDB is running"})
    except Exception as e:
        return jsonify({"status":"MongoDB is not running", "error": str(e)})

if __name__=='__main__':
    mqtt_thread = threading.Thread(target=start_mqtt,daemon=True)
    mqtt_thread.start()
    app.run(host='0.0.0.0', port=5002, debug=True)            

 * Serving Flask app '__main__'
 * Debug mode: on


/tmp/ipykernel_411335/2908328419.py:23: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://10.2.133.229:5002
Press CTRL+C to quit
 * Restarting with stat


Connected with result code 0


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/ayush-sm/my-subjects/Intro to IOT/iotmidsub/mqttfolder/.venv/lib/python3.13/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
    ~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/home/ayush-sm/my-subjects/Intro to IOT/iotmidsub/mqttfolder/.venv/lib/python3.13/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
    ~~~~~~~~~~~~~~^^^^^^
  File "/home/ayush-sm/my-subjects/Intro to IOT/iotmidsub/mqttfolder/.venv/lib/python3.13/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/ayush-sm/my-subjects/Intro to IOT/iotmidsub/mqttfolder/.venv/lib/python3.13/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
    ~~~~~~~~~~~~~~~~~^^
  File "/home/ayush-sm/my-subjects/

SystemExit: 1